# Assignment 2 : Bias in Data Assignment
The goal of this assignment is to explore the concept of bias through data on Wikipedia articles - specifically, articles on political figures from a variety of countries. For this assignment, you will combine a dataset of Wikipedia articles with a dataset of country populations, and use a machine learning service called ORES to estimate the quality of each article.

In [96]:
import pandas as pd
import requests
import numpy as np
import json

## Data cleaning

In [115]:
# load dataframes
page_data_df = pd.read_csv("data/page_data.csv")
wpds_data_df = pd.read_csv("data/wpds_data.csv")

We first need to get rid of rows started with `template:` in the `page_data` dataset as these pages are not Wikipedia articles.

In [116]:
page_data_df = page_data_df[~(page_data_df["page"].str.startswith("Template:"))]
page_data_df

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568
...,...,...,...
47192,Yahya Jammeh,Gambia,807482007
47193,Lucius Fairchild,United States,807483006
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153
47195,Francis Fessenden,United States,807483270


Similarly, we need to exclude rows in all caps for column `Name` in the `wpds_data` dataset as these rows provide cumulative regional population counts rather than country-level counts. Side note: we still want to retain the cumulative rows in a separate dataFrame.

In [117]:
wpds_data_cumulative_df = wpds_data_df[(wpds_data_df['Name'].str.isupper())] # we still want to retain them
wpds_data_df = wpds_data_df[~(wpds_data_df['Name'].str.isupper())]
wpds_data_df

,FIPS,Name,Type,TimeFrame,Data (M),Population
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000
5,LY,Libya,Country,2019,6.891,6891000
6,MA,Morocco,Country,2019,35.952,35952000
7,SD,Sudan,Country,2019,43.849,43849000
...,...,...,...,...,...,...
229,WS,Samoa,Country,2019,0.200,200000
230,SB,Solomon Islands,Country,2019,0.715,715000
231,TO,Tonga,Country,2019,0.099,99000
232,TV,Tuvalu,Country,2019,0.010,10000


## Requesting ORES to gather article quality predictions
We are using [ORES](http://google.com) to get the predicted quality score for each article in the `page_data` dataset. 

In [118]:
# first define our headers and create function call to call the ORES apis
req_headers = {
    "User-Agent": "https://github.com/jell0wed",
    "From": "jepoisso@uw.edu"
}

def api_call(endpoint, params):
    call = requests.get(endpoint.format(**params), headers=req_headers)
    resp = call.json()
    return resp

In [122]:
ores_articlequality_endpoint = "https://ores.wikimedia.org/v3/scores/{context}/{revid}/{model}"
ores_articlequality_params = {
    "context": "enwiki",
    "revid": None,
    "model": "articlequality"
}

i = 0
def fetch_ores_score(rev_id):
    global i
    
    if i < 10:
        ores_articlequality_params["revid"] = rev_id
        resp = api_call(ores_articlequality_endpoint, ores_articlequality_params)
        article_quality_dict = resp['enwiki']['scores'][str(rev_id)]['articlequality']
        if 'score' in article_quality_dict:
            
            return json.dumps(article_quality_dict['score']['probability'])
        i += 1
        print("processed %d" % (i))
    return ""

# do not uncomment the following lines, this took 50m to run, data was pre-processed in `data/ores_data.csv`
page_data_df["ores_score"] = page_data_df["rev_id"]
page_data_df["ores_score"] = page_data_df["ores_score"].apply(fetch_ores_score)
page_data_df.to_csv("ores_data.csv")

355319463
393276188
393822005
395521877
395526568
401577829
442937236
448555418
470173494
477962574
492060822
492964343
498683267
502721672
516633096
521986779
532253442
543225630
545936100
546364151
549300521
550682925
550953646
559553872
559788982
560758943
561744402
564873005
565745353
565745365
565745375
566504165
573710096
574571582
576988466
585894477
592289232
595693452
596181202
598819900
601122766
601127343
614786300
623004627
623334577
624468970
625509885
626606789
627001041
627051151
627432937
627547024
628261896
628268705
628270736
628312759
628379479
628563978
628619000
628766656
628988952
629562076
629818376
630396351
630396786
630704768
631437331
631581752
632008524
632261377
632447328
633612729
634032715
635240253
635814126
636911471
637801253
638214719
638362866
638377138
638566016
638571205
638599355
639021339
639061161
639471171
640014648
640214913
640826254
641422326
643410335
643746000
643932216
643932220
643932225
643932226
643932239
643932242
644024203
644041882


710675515
710676772
710677703
710679320
710684508
710688059
710688917
710690814
710699672
710715953
710741222
710753764
710759572
710786534
710791043
710795636
710799910
710802886
710803018
710803496
710803787
710807598
710809460
710812022
710813119
710816604
710828020
710831684
710844112
710844191
710866616
710870920
710871074
710882367
710891170
710891295
710911346
710911349
710912869
710912925
710924239
710929355
710930649
710930776
710930832
710942830
710950386
710954225
710954270
710961938
710962188
710962198
710962215
710962243
710963829
710969632
710973406
710983727
710985648
710990363
710995551
710998579
710999481
710999695
710999827
711000398
711010347
711014091
711014199
711020711
711029847
711103077
711103096
711104781
711112498
711115345
711117871
711137826
711137887
711140161
711147128
711151461
711152854
711153984
711157138
711158503
711160096
711160193
711160287
711160441
711160540
711160623
711160631
711160676
711170038
711170881
711177510
711197366
711197433
711210443


717925806
717927381
717927429
717927538
717928919
717930558
717932963
717935417
717935677
717937030
717937667
717939366
717945648
717958045
717961259
717962073
717972079
717977304
717999176
718002836
718004725
718019701
718025155
718025343
718026752
718029381
718036431
718036963
718037112
718041739
718052714
718058196
718079051
718079062
718079230
718081937
718090116
718098566
718100580
718104600
718111958
718121846
718126121
718131109
718132526
718134464
718146373
718160453
718161188
718170043
718172201
718185611
718187662
718187743
718204974
718205319
718205760
718206052
718206451
718206692
718210388
718221939
718222049
718225373
718231552
718248321
718250607
718254770
718256933
718257706
718265809
718271001
718292745
718299461
718342724
718345769
718346451
718351542
718362010
718362588
718364221
718364462
718366348
718382045
718383950
718384584
718384771
718388780
718392987
718400079
718400488
718400728
718400742
718402565
718405459
718407258
718412196
718414096
718414509
718418787


724550483
724552501
724553500
724554724
724554948
724557814
724564986
724575092
724579380
724579447
724590862
724593705
724606038
724610329
724612345
724620220
724621036
724625467
724628450
724630451
724632104
724635347
724641904
724647215
724659629
724659838
724661522
724663004
724671827
724674137
724678615
724678968
724686059
724691268
724695388
724699172
724703530
724706545
724707072
724707096
724740096
724758025
724761099
724767246
724771130
724803733
724803914
724804717
724807659
724812020
724818337
724823670
724827562
724830553
724830612
724835152
724836169
724837476
724838717
724839540
724846656
724848015
724848031
724848646
724849330
724849395
724851479
724854353
724859665
724861210
724861783
724862716
724869427
724871095
724872220
724872861
724874056
724874883
724874973
724875687
724878019
724879669
724880477
724880556
724881596
724882255
724882834
724888837
724891356
724891592
724896019
724897676
724898277
724898287
724899193
724900067
724900868
724900968
724919065
724919107


734122106
734122187
734125673
734130086
734136488
734136897
734138330
734178880
734215262
734221994
734260518
734313660
734323562
734344899
734346024
734351971
734353865
734366909
734367264
734367676
734368338
734390021
734393180
734399076
734399979
734419378
734427682
734437805
734474605
734484063
734484672
734484792
734485661
734486299
734498999
734511726
734521419
734527671
734540809
734550538
734611180
734619908
734620074
734620501
734640011
734648115
734662009
734668075
734672773
734692582
734695596
734746670
734747650
734756202
734768346
734768903
734771077
734799659
734801573
734813884
734832431
734840879
734847782
734858030
734858064
734865369
734883244
734883389
734899773
734936505
734957625
734962885
734969465
734972535
734983230
734986554
734999538
735021957
735024132
735030447
735031292
735045160
735048979
735089772
735092037
735098018
735099306
735110194
735147671
735157673
735167828
735174171
735190518
735206432
735232646
735263515
735281835
735296216
735296668
735302807


...

802788214
802789644
802789728
802792299
802792491
802792821
802794993
802795254
802795410
802795569
802800084
802800899
802801512
802802085
802805085
802807681
802808387
802808529
802809681
802809688
802811272
802811684
802813656
802813951
802815802
802816894
802817420
802817887
802818350
802818786
802819262
802820876
802821396
802822017
802823064
802824483
802825026
802826483
802827598
802828040
802830023
802830603
802832934
802833118
802833516
802834112
802834867
802834939
802835289
802835310
802835421
802835604
802835768
802835899
802835977
802837499
802839168
802839303
802839704
802841584
802841590
802841606
802841849
802842116
802843828
802844034
802844094
802844100
802844116
802844739
802845430
802846759
802847128
802847173
802850235
802850619
802851430
802852054
802852148
802853349
802853557
802854632
802854777
802855076
802855209
802855423
802855428
802855951
802856852
802856885
802858382
802859459
802859486
802859738
802860340
802860970
802861534
802862968
802863065
802864450


804091882
804092712
804093565
804093651
804093764
804094659
804095077
804095754
804095790
804099267
804099373
804100365
804101492
804103114
804104355
804104931
804105438
804105709
804105984
804106203
804106598
804106609
804106821
804107345
804108433
804109224
804109267
804111013
804111132
804112392
804112500
804112819
804113674
804114360
804115829
804116289
804118645
804118985
804120072
804120185
804122433
804122505
804123422
804124368
804124946
804125853
804126930
804127803
804127877
804128734
804128826
804129698
804130200
804130440
804130858
804131153
804131272
804131604
804131643
804131948
804132298
804132518
804132817
804133123
804133555
804134156
804136596
804136732
804138803
804139010
804139208
804139258
804139515
804139730
804139800
804139868
804140099
804140159
804140314
804140447
804142156
804142444
804142943
804143236
804143605
804145524
804146604
804147118
804147288
804148964
804149674
804151581
804151598
804152406
804152925
804153459
804154484
804155669
804156184
804157350


805241522
805241904
805242667
805243380
805243586
805244157
805244405
805244800
805245515
805246276
805247562
805248573
805248828
805249932
805250547
805250645
805251974
805252877
805253380
805253461
805253801
805254568
805259105
805259264
805259988
805260085
805260550
805260781
805260821
805261222
805261228
805263257
805263542
805264292
805264393
805266750
805267943
805267975
805268117
805268515
805270130
805273109
805273173
805273312
805274848
805275246
805276486
805276516
805276523
805276573
805276645
805276799
805276864
805276899
805277093
805277771
805278526
805279692
805279807
805280492
805281098
805281207
805281416
805281788
805282595
805283153
805284209
805286414
805286973
805287382
805287852
805287882
805287917
805288679
805289045
805289210
805289323
805289352
805289936
805290284
805291334
805291640
805292129
805293256
805294574
805296539
805298454
805299700
805301599
805303117
805304477
805305142
805305263
805305462
805306036
805306238
805306390
805307345
805308710
805313523


806453846
806455331
806455685
806456329
806457259
806460468
806460970
806461725
806461795
806462613
806463880
806464279
806464775
806466645
806467016
806468076
806468208
806469298
806469333
806470353
806470589
806470880
806471954
806472203
806474260
806475926
806476255
806477511
806478058
806478265
806478579
806478787
806478889
806479527
806479564
806481036
806481458
806481551
806482808
806483731
806487145
806489625
806490350
806490518
806490571
806491339
806491599
806491618
806491860
806492485
806492530
806492920
806493000
806493632
806493652
806493972
806494765
806496321
806497393
806497540
806498083
806498397
806499019
806499067
806499431
806499548
806500735
806500816
806501595
806501713
806502951
806502993
806503089
806503132
806503196
806503217
806503337
806503372
806503445
806504184
806504904
806506945
806507288
806507554
806507568
806507570
806508319
806508985
806509038
806509190
806509643
806510585
806510922
806511942
806512047
806512259
806513371
806513415
806513546
806513820


806925410
806927054
806928362
806928457
806928471
806929601
806931313
806931367
806931377
806931864
806932052
806934027
806936953
806938514
806939531
806941754
806941972
806943108
806947297
806947596
806948506
806948949
806949962
806950167
806951645
806952259
806952883
806952931
806955709
806955889
806956559
806956603
806957759
806958416
806958982
806959295
806959484
806961306
806962327
806964376
806964521
806964950
806965618
806972002
806973935
806975039
806975273
806975975
806976735
806977245
806977351
806977386
806978501
806978659
806978680
806980105
806980973
806981371
806982443
806983342
806983705
806984938
806987838
806988271
806988445
806990454
806991055
806991412
806992091
806994136
806994231
806994489
806994641
806994807
806995404
806995881
806996449
806996844
806997382
806997998
806998653
806998805
806999119
806999198
806999239
806999392
806999606
806999753
806999950
807000038
807000149
807000274
807000457
807000784
807000951
807001015
807001216
807001460
807003284
807004352


In [113]:
page_data_df = pd.read_csv("ores_data.csv")
page_data_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,page,country,rev_id,ores_scopres,ores_scores
0,0,0,0,0,0,1,Bir I of Kanem,Chad,355319463,NaN,0
1,1,1,1,1,1,10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,NaN,1
2,2,2,2,2,2,12,Yos Por,Cambodia,393822005,NaN,2
3,3,3,3,3,3,23,Julius Gregr,Czech Republic,395521877,NaN,3
4,4,4,4,4,4,24,Edvard Gregr,Czech Republic,395526568,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...
46696,46696,46696,46696,46696,46696,47192,Yahya Jammeh,Gambia,807482007,NaN,46696
46697,46697,46697,46697,46697,46697,47193,Lucius Fairchild,United States,807483006,NaN,46697
46698,46698,46698,46698,46698,46698,47194,Fahd of Saudi Arabia,Saudi Arabia,807483153,NaN,46698
46699,46699,46699,46699,46699,46699,47195,Francis Fessenden,United States,807483270,NaN,46699
